In [47]:
!kubectl get -n kafka-main-cluster kafkaconnectors debezium-connector-postgres -o yaml | yq

apiVersion: kafka.strimzi.io/v1beta2
kind: KafkaConnector
metadata:
  annotations:
    kubectl.kubernetes.io/last-applied-configuration: |
      {"apiVersion":"kafka.strimzi.io/v1beta2","kind":"KafkaConnector","metadata":{"annotations":{},"labels":{"strimzi.io/cluster":"my-connect"},"name":"debezium-connector-postgres","namespace":"kafka-main-cluster"},"spec":{"class":"io.debezium.connector.postgresql.PostgresConnector","config":{"converter.type":"avro","database.dbname":"airflow","database.hostname":"postgres-postgresql.postgres.svc","database.password":"${secrets:kafka-main-cluster/postgres-user-debezium:password}","database.port":5432,"database.user":"${secrets:kafka-main-cluster/postgres-user-debezium:username}","key.converter":"io.confluent.connect.avro.AvroConverter","key.converter.schema.registry.url":"http://main-registry-schema-registry.kafka-main-cluster.svc:8081","plugin.name":"decoderbufs","schema.history.internal.kafka.bootstrap.servers":"main-kafka-bootstrap:9092","schema

# Get connector error message

In [49]:
!kubectl get -n kafka-main-cluster kafkaconnectors debezium-connector-postgres -o jsonpath='{.status.connectorStatus.tasks[0].trace}' 

In [50]:
!kubectl -n kafka-main-cluster get secrets postgres-user-debezium -o yaml | yq

apiVersion: v1
data:
  password: ZGViZXppdW0=
  username: ZGViZXppdW0=
kind: Secret
metadata:
  annotations:
    replicator.v1.mittwald.de/replicated-at: "2024-02-07T10:54:26Z"
    replicator.v1.mittwald.de/replicated-from-version: "8759"
    replicator.v1.mittwald.de/replicated-keys: password,username
  creationTimestamp: "2024-02-07T10:54:26Z"
  name: postgres-user-debezium
  namespace: kafka-main-cluster
  resourceVersion: "8760"
  uid: bf4de92f-9979-4253-ac84-eed623ce3a53
type: Opaque


In [42]:
!curl -s -H "Accept:application/json" http://my-connect-connect-api.kafka-main-cluster.svc:8083/connector-plugins/source/config/validate | jq

{
  "error_code": 405,
  "message": "HTTP 405 Method Not Allowed"
}
